In [54]:
import pandas as pd
import numpy as np
import csv

rootdir = os.getcwd()

In [55]:
league = 'SBC'
season = 2040
curr_date = '2040-05-28'
open_date = '2040-03-29'
league_al_name = 'AL' 
league_nl_name = 'NL'
league_id = 145

# NBC Tournament only. This needs to be updated once annually
tourney_start = '2023-07-04'
tourney_end = '2023-07-25'

In [56]:
def comp_balance(league):
    team_history = pd.read_csv(rootdir + '/csv/team_history.csv')
    team_history_financials = pd.read_csv(rootdir + '/csv/team_history_financials.csv')
    team_history = team_history[(team_history['year'] == (season - 1)) & 
                            (team_history['league_id'] == league_id)].set_index('team_id')

    team_history = team_history[['name', 'nickname', 'made_playoffs']]

    team_history_financials = team_history_financials[(team_history_financials['year'] == (season - 1)) & 
                            (team_history_financials['league_id'] == league_id)].set_index('team_id')

    temp = pd.merge(team_history, team_history_financials, how = 'left', left_index = True, right_index = True)
    
    comp_bal = pd.DataFrame()
    comp_bal['Team'] = temp['name'] + ' ' + temp['nickname']
    comp_bal['Total Revenue (millions)'] = round((temp['gate_revenue'] + \
                                temp['season_ticket_revenue'] + \
                                temp['media_revenue'] + \
                                temp['merchandising_revenue'] + \
                                temp['playoff_revenue'])/1000000,1)
    comp_bal['Media Revenue (millions)'] = temp['media_revenue'] / 1000000
    comp_bal['Media Revenue %'] = round(((comp_bal['Media Revenue (millions)'])/comp_bal['Total Revenue (millions)'])*100,1)
    comp_bal['Other Revenue %'] = 100 - comp_bal['Media Revenue %']
    comp_bal['Market Size'] = temp['market']
    comp_bal['Playoffs'] = np.where(temp['made_playoffs'] == 0, 'N','Y')
    
    return comp_bal


In [58]:
def teams(league):
    teams = pd.read_csv(rootdir + '/csv/teams.csv')[['name', 'nickname','team_id','sub_league_id', 'division_id']]
    teams = teams[teams['team_id'] < 80]
    teams['league'] = np.where(teams['sub_league_id'] == 0, "AL", "NL")
    teams['division'] = np.select(
    [
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 2),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 2)
    ],
    [
        'East',
        'Central',
        'West',
        'East',
        'Central',
        'West'
    ],
    default = 'Unknown'
    )

    teams['team'] = teams['name'] + ' ' + teams['nickname']

    return teams[['team', 'league', 'division']].set_index(teams['team_id'])

In [94]:
home_games = pd.read_csv(rootdir + '/csv/games.csv')
home_games['date'] =  pd.to_datetime(home_games['date'], format='%Y/%m/%d')
home_games = home_games[(home_games['league_id'] == leagues[league][0]) & 
                        (home_games['game_type'] == 0) &
                        (home_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
home_games['home_id'] = home_games['home_team']
home_games['home_runs'] = home_games['runs1']

home_games = home_games[['home_id','date', 'home_runs', 'innings']].set_index(home_games['game_id'])


away_games = pd.read_csv(rootdir + '/csv/games.csv')
away_games['date'] =  pd.to_datetime(away_games['date'], format='%Y/%m/%d')
away_games = away_games[(away_games['league_id'] == leagues[league][0]) & 
                        (away_games['game_type'] == 0) &
                        (away_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
away_games['away_id'] = away_games['away_team']
away_games['away_runs'] = away_games['runs0']

away_games = away_games[['away_id','date', 'away_runs', 'innings']].set_index(away_games['game_id'])

games_columns = away_games.columns.difference(home_games.columns)

games = home_games.merge(away_games[games_columns], on = 'game_id').merge(
    teams(league), how = 'left', left_on = 'home_id', right_on = 'team_id').rename(
    columns={'team':'home_team', 'league':'home_league', 'division':'home_division'}).merge(
    teams(league), how = 'left', left_on = 'away_id', right_on = 'team_id').rename(
    columns={'team':'away_team', 'league':'away_league', 'division':'away_division'})

all_games = games.copy()
all_games['hw'] = np.where(all_games['home_runs'] > all_games['away_runs'],1,0)
all_games['hl'] = np.where(all_games['home_runs'] < all_games['away_runs'],1,0)
all_games['aw'] = np.where(all_games['home_runs'] < all_games['away_runs'],1,0)
all_games['al'] = np.where(all_games['home_runs'] > all_games['away_runs'],1,0)

all_games['hd'] = all_games.home_runs - all_games.away_runs
all_games['ad'] = all_games.away_runs - all_games.home_runs

all_games['hdw'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hdl'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['adw'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['adl'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hcw'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hcl'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['acw'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['acl'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hiw'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hil'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['aiw'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['ail'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hexw'] = np.where((all_games.innings > 9) & (all_games['home_runs'] > all_games['away_runs']),1,0)
all_games['hexl'] = np.where((all_games.innings > 9) & (all_games['home_runs'] < all_games['away_runs']),1,0)
all_games['aexw'] = np.where((all_games.innings > 9) & (all_games['home_runs'] < all_games['away_runs']),1,0)
all_games['aexl'] = np.where((all_games.innings > 9) & (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['h1rw'] = np.where(all_games.home_runs - all_games.away_runs == 1,1,0)
all_games['h1rl'] = np.where(all_games.away_runs - all_games.home_runs == 1,1,0)
all_games['a1rw'] = np.where(all_games.away_runs - all_games.home_runs == 1,1,0)
all_games['a1rl'] = np.where(all_games.home_runs - all_games.away_runs == 1,1,0)

all_games['h2rw'] = np.where(all_games.home_runs - all_games.away_runs == 2,1,0)
all_games['h2rl'] = np.where(all_games.away_runs - all_games.home_runs == 2,1,0)
all_games['a2rw'] = np.where(all_games.away_runs - all_games.home_runs == 2,1,0)
all_games['a2rl'] = np.where(all_games.home_runs - all_games.away_runs == 2,1,0)

all_games['h3rw'] = np.where(all_games.home_runs - all_games.away_runs == 3,1,0)
all_games['h3rl'] = np.where(all_games.away_runs - all_games.home_runs == 3,1,0)
all_games['a3rw'] = np.where(all_games.away_runs - all_games.home_runs == 3,1,0)
all_games['a3rl'] = np.where(all_games.home_runs - all_games.away_runs == 3,1,0)

all_games['h4prw'] = np.where(all_games.home_runs - all_games.away_runs > 3 ,1,0)
all_games['h4prl'] = np.where(all_games.away_runs - all_games.home_runs > 3,1,0)
all_games['a4prw'] = np.where(all_games.away_runs - all_games.home_runs > 3,1,0)
all_games['a4prl'] = np.where(all_games.home_runs - all_games.away_runs > 3,1,0)

In [95]:
home_results = all_games.copy()
home_game_cols = [col for col in home_results if col.startswith('h')]
home_results = home_results[home_game_cols]

away_results = all_games.copy()
away_game_cols = [cols for cols in away_results if cols.startswith('a')]
away_results = away_results[away_game_cols]

In [96]:
home_beta = home_results.groupby(['home_team']).sum()
home_beta['hw%'] = round((home_beta['hw'] / (home_beta['hw'] + home_beta['hl'])*100),1)
home_beta = home_beta.drop(columns=['home_runs','home_id'])
#home_beta.rename(columns={'home_team':'team'}, inplace=True)

away_beta = away_results.groupby(['away_team']).sum()
away_beta['aw%'] = round((away_beta['aw'] / (away_beta['aw'] + away_beta['al'])*100),1)
away_beta = away_beta.drop(columns=['away_runs','away_id'])
#away_beta.rename(columns={'away_team':'team'}, inplace=True)

In [97]:
home_beta.columns = home_beta.columns.str[1:]
away_beta.columns = away_beta.columns.str[1:]

In [98]:
results = home_beta.add(away_beta, axis ='columns')
results['w%'] = results['w'] / (results['w'] + results['l'])

In [99]:
home_beta

,w,l,d,dw,dl,cw,cl,iw,il,exw,exl,1rw,1rl,2rw,2rl,3rw,3rl,4prw,4prl,w%
home_team,,,,,,,,,,,,,,,,,,,,
Arizona Athletics,21,5,54,19,4,19,4,2,1,3,0,6,0,0,0,3,2,12,3,80.8
Atlanta Braves,14,8,19,14,8,14,8,0,0,0,1,1,3,3,0,4,3,6,2,63.6
Baltimore Orioles,12,12,-27,11,11,11,11,1,1,2,0,7,3,1,1,1,2,3,6,50.0
Boston Red Sox,11,14,-4,11,11,11,11,0,3,1,3,2,1,3,4,0,5,6,4,44.0
Carolina Swamp Foxes,19,8,47,15,7,15,7,4,1,4,1,4,3,6,1,1,2,8,2,70.4
Chicago Cubs,6,22,-61,6,22,6,22,0,0,0,2,1,3,2,5,2,6,1,8,21.4
Cincinnati Reds,11,15,-22,11,13,11,13,0,2,1,4,5,5,2,3,1,1,3,6,42.3
Cleveland Spiders,16,10,51,14,10,14,10,2,0,0,1,3,3,1,4,3,1,9,2,61.5
Detroit Tigers,20,6,60,19,5,19,5,1,1,2,1,5,1,4,1,3,4,8,0,76.9


In [100]:
sbc_comp_bal = comp_balance('SBC')

sbc_comp_bal.to_excel(rootdir + '/' + str(season) + '/comp balance.xlsx')
results.to_excel(rootdir + '/' + str(season) + '/full standings.xlsx')
home_beta.to_excel(rootdir + '/' + str(season) + '/home standings.xlsx')
away_beta.to_excel(rootdir + '/' + str(season) + '/away standings.xlsx')